<a href="https://colab.research.google.com/github/Pranav-Bhatlapenumarthi/Deep_Learning/blob/main/CNN_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1) Setup and Imports

We only use:
- `numpy` (math & arrays)
- `matplotlib` (optional plots)
- `urllib.request`, `tarfile`, `pickle` (to download & parse CIFAR‑10)

In [1]:
import os, tarfile, pickle, urllib.request, time, math, random
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

# 2) CIFAR‑10 Loader (from raw batches)

CIFAR‑10 provides 60k 32×32 color images in 10 classes.  We download the original python version and parse it.

In [ ]:
CIFAR_URL = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
CIFAR_TAR = "cifar-10-python.tar.gz"
CIFAR_DIR = "cifar-10-batches-py"

def download_cifar10():
    if not os.path.exists(CIFAR_TAR):
        print("Downloading CIFAR-10...")
        urllib.request.urlretrieve(CIFAR_URL, CIFAR_TAR)
    if not os.path.exists(CIFAR_DIR):
        print("Extracting CIFAR-10...")
        with tarfile.open(CIFAR_TAR) as tar:
            tar.extractall()
    print("CIFAR-10 ready.")

def load_batch(filepath):
    with open(filepath, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
        data = batch[b'data']    # shape (10000, 3072)
        labels = batch[b'labels']  # list length 10000
        data = data.reshape(-1, 3, 32, 32).astype(np.float32)
        labels = np.array(labels, dtype=np.int64)
        return data, labels

def load_cifar10():
    download_cifar10()
    xs, ys = [], []
    for i in range(1,6):
        Xb, yb = load_batch(os.path.join(CIFAR_DIR, f"data_batch_{i}"))
        xs.append(Xb); ys.append(yb)
    X_train = np.concatenate(xs, axis=0)
    y_train = np.concatenate(ys, axis=0)
    X_test, y_test = load_batch(os.path.join(CIFAR_DIR, "test_batch"))
    return (X_train, y_train), (X_test, y_test)

def per_channel_normalize(X, mean=None, std=None):
    # X: (N, C, H, W)
    if mean is None: mean = X.mean(axis=(0,2,3), keepdims=True)
    if std  is None: std  = X.std(axis=(0,2,3), keepdims=True) + 1e-7
    return (X - mean) / std, mean, std

def show_samples(X, y, n=8):
    # quick grid view
    fig, axes = plt.subplots(1, n, figsize=(n*2, 2))
    for i in range(n):
        img = X[i].transpose(1,2,0)
        axes[i].imshow(img.astype(np.float32))
        axes[i].axis('off')
    plt.show()

(X_train, y_train), (X_test, y_test) = load_cifar10()
X_train, mean_c, std_c = per_channel_normalize(X_train)
X_test, _, _ = per_channel_normalize(X_test, mean_c, std_c)

print("Train:", X_train.shape, y_train.shape)
print("Test: ", X_test.shape, y_test.shape)

show_samples(X_train.copy(), y_train, n=8)